# Projet Uber

**Projet**  
L'un des principaux problèmes rencontrés par l'équipe d'Uber est que parfois les chauffeurs ne sont pas là lorsque les utilisateurs en ont besoin. Par exemple, un utilisateur peut se trouver dans le quartier financier de San Francisco alors que les chauffeurs Uber recherchent des clients à Castro.  
  
Par conséquent, l'équipe de données d'Uber aimerait travailler sur un projet dans lequel **son application recommanderait des zones chaudes dans les grandes villes à n'importe quel moment de la journée.**

**Objectifs**  
1) Travailler sur un algorythme pour trouver les zones chaudes  
2) Visualiser les résultats sur un tableau de bord

# Importation des librairies

In [1]:

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px 

from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score

# Chargement du dataset

In [2]:
uber = pd.read_csv('uber-raw-data-apr14.csv')

In [3]:
uber.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


Ce dataset donne la géolocalisation (latitude et longitude) de chaque course Uber, pour tout le mois d'avril.

# Analyse du dataset

In [4]:
print(uber.shape)
print()

print(uber.describe())
print()

print(uber.info())
print()

print(uber.isna().mean())

(564516, 4)

                 Lat            Lon
count  564516.000000  564516.000000
mean       40.740005     -73.976817
std         0.036083       0.050426
min        40.072900     -74.773300
25%        40.722500     -73.997700
50%        40.742500     -73.984800
75%        40.760700     -73.970000
max        42.116600     -72.066600

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564516 entries, 0 to 564515
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date/Time  564516 non-null  object 
 1   Lat        564516 non-null  float64
 2   Lon        564516 non-null  float64
 3   Base       564516 non-null  object 
dtypes: float64(2), object(2)
memory usage: 17.2+ MB
None

Date/Time    0.0
Lat          0.0
Lon          0.0
Base         0.0
dtype: float64


In [5]:
uber['Base'].value_counts()

B02682    227808
B02598    183263
B02617    108001
B02512     35536
B02764      9908
Name: Base, dtype: int64

Pour avoir une nouvelles approche pour travailler sur ce dataset, nous allons créer de nouvelles variables.  
Pour chaque réservation Uber, nous allons ajouter les variables "mois", "heure", "minute" et "jour".  
L'objectid est simple, il sera possible de faire des cluster selon le jour du mois ou selon une certaine heure de la journée.

In [6]:
# Utilisation de datetime pour obtenir des features mois, heure, minute et jour
uber['Date/Time'] = pd.to_datetime(uber['Date/Time'])
uber.loc[:,'Month'] = uber['Date/Time'].dt.month
uber.loc[:,'Heure'] = uber['Date/Time'].dt.hour
uber.loc[:,'Minute'] = uber['Date/Time'].dt.minute
uber.loc[:,'Jour'] = uber['Date/Time'].dt.day

In [7]:
uber.head()

,Date/Time,Lat,Lon,Base,Month,Heure,Minute,Jour
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,4,0,11,1
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,4,0,17,1
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,4,0,21,1
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,4,0,28,1
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,4,0,33,1


Pour le début, nous allons se focaliser sur un jour du mois d'avril. Je vais travailler sur le jour 1 du mois d'avril.  
Deux modèle de machine learnng seront tester :  
-DBSCAN
-KMeans

# Première analyse sur 1 jour du mois 

In [8]:
# On selectionne que le jour 1 du mois d'avril
df= uber[uber['Jour'] == 1 ]
df.head()

,Date/Time,Lat,Lon,Base,Month,Heure,Minute,Jour
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,4,0,11,1
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,4,0,17,1
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,4,0,21,1
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,4,0,28,1
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,4,0,33,1


In [9]:
df.shape

(14546, 8)

In [10]:
# Représentation graphique des demandes Uber
fig = px.scatter_mapbox(df, lat="Lat", lon="Lon",color="Minute"
                  ,color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10, title="Représention de chaques réservation Uber en 1 heure")
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show('iframe')

# DBSCAN MODEL (Sur le jour 1 du mois d'avril)

**DBSCAN**  
Pour cet algorithme, pas besoin de lui spécifier le nombre de cluster à l'avance.  
Il utilise le concepte de densité pour affecter des observation à un agrégat.  
Pour tuiliser cet algorythme, je vais jouer sur 2 paramètres :  
- eps (epsilone) : la distance entre un point et les limites du voisinage.
- min_sample : le nombre minimum de points necessaire pour definir un échantillon de base.

In [11]:
#On sélection que les données necessaire pour le fonctionnement du modèle 
df_model = df.drop(columns = ['Date/Time', 'Month', 'Minute', 'Base', 'Jour', 'Heure'])
df_model.head()

,Lat,Lon
0,40.7690,-73.9549
1,40.7267,-74.0345
2,40.7316,-73.9873
3,40.7588,-73.9776
4,40.7594,-73.9722


In [12]:
# Boucle for pour tester deux hypers paramètres du modèle DBSCAN

eps = [0.01, 0.02, 0.05, 0.1]
min_samples = [0.5, 1, 5, 10, 15]

for e in eps:
    for m in min_samples:
        model = DBSCAN(eps = e , min_samples = m, metric="euclidean", algorithm="brute")
        model.fit(df_model)
        print(e,m)
        print(np.unique(model.labels_))
        print()

0.01 0.5
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168]

0.01 1
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72

In [13]:
# On fit le model sur nos données 
model = DBSCAN(eps = 0.01 , min_samples = 15, metric="euclidean", algorithm="brute")
model.fit(df_model)

DBSCAN(algorithm='brute', eps=0.01, min_samples=15)

In [14]:
# On récupère la prédiction de modèle
df_model['cluster'] = model.labels_
df_model.head()

,Lat,Lon,cluster
0,40.7690,-73.9549,0
1,40.7267,-74.0345,0
2,40.7316,-73.9873,0
3,40.7588,-73.9776,0
4,40.7594,-73.9722,0


In [15]:
df_model['cluster'].value_counts()

 0    13362
-1      434
 3      309
 1      306
 2      118
 4       17
Name: cluster, dtype: int64

In [16]:
fig = px.scatter_mapbox(
df_model[df_model.cluster != -1], 
lat="Lat", 
lon="Lon",
color="cluster",
mapbox_style="carto-positron")
fig.show("iframe")

**Résultats**  
J'ai choisi eps et min_sample de façon à ne former que 5 clusters. On peut remarquer que les cluster ne sont pas homogène. Que le gros cluster bleu (sur le graphique) n'est pas assez précis pour le besoin du projet.  
**Perspective**  
changer les hypers paramètres pour obtenir un résultat répondant au besoin

# Kmeans MODEL (Sur le jour 1 du mois d'avril)

**Kmeans**  
K-means est une technique de partitionnement, qui sépare les observations en k clusters. Où chaque observation est considérée comme appartenant au groupe où la moyenne est la plus proche selon une distance donnée.

In [17]:
df_model = df.drop(columns = ['Date/Time', 'Month', 'Minute', 'Base', 'Jour', 'Heure'])
df_model.head()

,Lat,Lon
0,40.7690,-73.9549
1,40.7267,-74.0345
2,40.7316,-73.9873
3,40.7588,-73.9776
4,40.7594,-73.9722


**Méthode Elbow**  
La méthodes Elbow essaie de voir si chaque point de données d'un cluster est proche les uns des autres.

In [18]:
# Application de la méthode Elbow avec une boucle for. L'objectif est de modifier le nbr de cluster de 1 à 19 pour connaître l'inertie pour chaque valeur de K

wcss =  []
k = []
for i in range (1,20): 
    kmeans = KMeans(n_clusters= i, init = "k-means++", random_state = 0)
    kmeans.fit(df_model)
    wcss.append(kmeans.inertia_)
    k.append(i)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))

WCSS for K=1 --> 59.44371542664515
WCSS for K=2 --> 37.71280837380052
WCSS for K=3 --> 28.32134393189509
WCSS for K=4 --> 20.89571128178056
WCSS for K=5 --> 15.875392812992542
WCSS for K=6 --> 12.822573252255077
WCSS for K=7 --> 10.897900139268302
WCSS for K=8 --> 9.576220172129554
WCSS for K=9 --> 8.635591434579691
WCSS for K=10 --> 7.7224400146449454
WCSS for K=11 --> 6.928455457530976
WCSS for K=12 --> 6.217536322131518
WCSS for K=13 --> 5.517676366475769
WCSS for K=14 --> 5.1977377842361765
WCSS for K=15 --> 4.838647041484005
WCSS for K=16 --> 4.549930753455824
WCSS for K=17 --> 4.312199109788333
WCSS for K=18 --> 4.08306278558196
WCSS for K=19 --> 3.801429864212174


In [19]:
# Création d'un DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Création d'une figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Création d'un axe et d'un titre
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

fig.show(renderer="iframe") # if using workspace

Avec la méthode Elbow, le nombre de cluster optimal serait compris entre 3 et 6

**Méthode Silhouette**  
La méthode silhouette essaie de déterminer à quelle distance se trouvent chaque groupe(et donc à quel point ils sont significatifs)

In [20]:
# Application de la méthode silhouette avec un boucle for. L'objectif est de modifier le nbr de cluster de 1 à 19 pour connaître la silhouette score pour chaque valeur de K

sil = []
k = []

for i in range (2,11): 
    kmeans = KMeans(n_clusters= i, init = "k-means++", random_state = 0)
    kmeans.fit(df_model)
    sil.append(silhouette_score(df_model, kmeans.predict(df_model)))
    k.append(i)
    print("Silhouette score for K={} is {}".format(i, sil[-1]))

Silhouette score for K=2 is 0.7263771291722336
Silhouette score for K=3 is 0.40773729148770177
Silhouette score for K=4 is 0.428847533402157
Silhouette score for K=5 is 0.44982133054459494
Silhouette score for K=6 is 0.4713777148995064
Silhouette score for K=7 is 0.4747723843351138
Silhouette score for K=8 is 0.43168475349058044
Silhouette score for K=9 is 0.43099296470182463
Silhouette score for K=10 is 0.4315161351762257


In [21]:
cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

fig = px.bar(data_frame=cluster_scores,  
             x=k, 
             y=cluster_scores.iloc[:, -1]
            )

fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score per cluster"
)


fig.show(renderer="iframe")

**Résultats**  
La méthode silhouette montre que 4 clusters est le nombre de cluster "idéal".  
Par la suite je vais donc travailler avec un nombre de cluster de 4

In [22]:
# On applique le modèle à nos données

kmeans = KMeans(n_clusters= 4, init = "k-means++", random_state = 0)
kmeans.fit(df_model)

KMeans(n_clusters=4, random_state=0)

In [24]:
#On récupère les coordonnées des centres de cluster
cluster_centers = kmeans.cluster_centers_

In [25]:
cluster_centers

array([[ 40.76782741, -73.96612107],
       [ 40.72198106, -73.99254926],
       [ 40.67683229, -73.77723341],
       [ 40.69598509, -74.20891925]])

In [26]:
# On créer un variable y, qui contient les prédiction du modèle
y = kmeans.fit_predict(df_model)
y

array([0, 1, 1, ..., 1, 1, 0])

In [27]:
df2 = pd.Series(y, name = 'pred')
df2.value_counts()

1    7477
0    6462
2     446
3     161
Name: pred, dtype: int64

In [28]:
# On ajoute la colonne y(prédiction) à notre dataframe de départ
df_model['cluster'] = y
df_model

,Lat,Lon,cluster
0,40.7690,-73.9549,0
1,40.7267,-74.0345,1
2,40.7316,-73.9873,1
3,40.7588,-73.9776,0
4,40.7594,-73.9722,0
...,...,...,...
554926,40.7219,-73.9920,1
554927,40.7261,-74.0027,1
554928,40.7364,-73.9926,1
554929,40.7149,-73.9405,1


Pour chaque coordonnée le modèle l'attribut à un cluster. Qui sera représenté dans le graphique suivant.

In [29]:
fig = px.scatter_mapbox(
df_model[df_model.cluster != -1], 
lat="Lat", 
lon="Lon",
color="cluster",
mapbox_style="carto-positron")
fig.show("iframe")

Sur ce graphique (modèle K-Means), on remarque que le centre ville est cette fois-ci composé de 2 clusters.  Le Cluster 1 (représenté en violet) regroupe sur 1 jour plus de 1000 demande que le cluster 0 (représenté en bleu).

# Conclusion sur cette première analyse : 

Pour cette première analyse, je me suis concentré sur le jour 1 du mois d'avril. On remarque que le modèle Kmeans est le modèle le plus approprié pour créer des clusters biens distincts et identifiables. Pour la suite du projet, je vais garder ce modèle. Nous allons faire une analyse sur l'ensemble du mois d'avril, en regroupant par jour de la semaine (Donc 7 jours)

# Analyse par jour de la semaine du mois d'Avil : 

In [30]:
df.head()

,Date/Time,Lat,Lon,Base,Month,Heure,Minute,Jour
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,4,0,11,1
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,4,0,17,1
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,4,0,21,1
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,4,0,28,1
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,4,0,33,1


In [31]:
# Création de la colonne Semaine

uber['Date/Time'] = pd.to_datetime(uber['Date/Time'])
uber.loc[:,'Semaine'] = uber['Date/Time'].dt.dayofweek
uber.head()

,Date/Time,Lat,Lon,Base,Month,Heure,Minute,Jour,Semaine
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,4,0,11,1,1
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,4,0,17,1,1
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,4,0,21,1,1
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,4,0,28,1,1
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,4,0,33,1,1


In [32]:
uber['Semaine'].value_counts()
# On a sept résultats corrspndant aux 7 jours de la semaine!

2    108631
1     91185
4     90303
3     85067
5     77218
0     60861
6     51251
Name: Semaine, dtype: int64

In [33]:
# On sélection seulement la semaine 1

uber_model = uber.loc[uber['Semaine']==1, ['Lat', 'Lon']]
uber_model.head()

,Lat,Lon
0,40.7690,-73.9549
1,40.7267,-74.0345
2,40.7316,-73.9873
3,40.7588,-73.9776
4,40.7594,-73.9722


In [34]:
uber_model.shape

(91185, 2)

le shape correspond bien au nombre de valeurs contenus dans le jour 1

In [35]:
uber['Semaine'].unique()

array([1, 2, 3, 4, 5, 6, 0], dtype=int64)

In [36]:
# On aplique le modèle dans une boucle for, pour qu'il donne des prédiction pour chaque jour de la semaine.

for j in uber['Semaine'].unique():
    uber_model = uber.loc[uber['Semaine']==j, ['Lat', 'Lon']]
    model = KMeans(n_clusters= 4, init = "k-means++", random_state = 0)
    model.fit(uber_model)
    
    uber_model.loc[:, 'Cluster'] = model.fit_predict(uber_model)
    
    fig = px.scatter_mapbox(
        uber_model[uber_model.Cluster != -1], 
        lat="Lat", 
        lon="Lon",
        color="Cluster",
        mapbox_style="carto-positron")
    fig.show(renderer="iframe")